In [1]:
import sys
import os
sys.path.append(os.path.abspath("../src"))

from src.infrastructure.config.settings import settings
from src.infrastructure.database.client import PostgresClient

from src.infrastructure.data.yfinance import YFinanceService
from src.domain.etl.services.extractor_service import ExtractorService
from src.domain.etl.services.enricher_service import EnricherService
from src.domain.etl.services.pipeline_service import FeaturePipelineService

from src.domain.strategies.engine import BacktestEngine
from src.domain.strategies.service import BacktestService
from src.domain.strategies.research_orchestrator import ResearchOrchestrator

import pandas as pd
from IPython.display import display, HTML

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 2000)
pd.set_option("display.max_colwidth", None)
display(HTML("<style>div.output_area pre {overflow-x: auto;}</style>"))

# 0) DB client
db_client = PostgresClient(
    db_url=settings.db_url,
    pool_size=settings.db_pool_size,
    max_overflow=settings.db_max_overflow,
    pool_timeout=settings.db_pool_timeout,
    pool_recycle=settings.db_pool_recycle,
    echo=settings.db_echo,
)
await db_client.init(create_schema=True)

# 1) Pipeline de features
yfs = YFinanceService()
extractor = ExtractorService(extractor=yfs)
enricher = EnricherService()
pipeline = FeaturePipelineService(extractor=extractor, transformer=enricher)

enriched_or_dict = await pipeline.start(
    symbols=None,   # default_symbols
    indicators=[],  # o lista concreta
)

# 2) Backtesting + persistencia
engine = BacktestEngine()
bt_service = BacktestService(engine)
orchestrator = ResearchOrchestrator(bt_service, db_client)

results = await orchestrator.run_and_persist(
    data=enriched_or_dict,
    strategy_names=None,
    strategy_params=None,
    timeframe="1h",
)

# 3) Mostrar mejor estrategia por símbolo (además de guardarla en la DB)
best_per_asset = {asset: reports[0] for asset, reports in results.items() if reports}

for asset, report in best_per_asset.items():
    print(f"\n=== {asset} ===")
    print(report.print_summary())


/Users/Mauro/miniconda3/envs/ar-trading-bot/lib/python3.12/site-packages/backtesting/_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

INFO:src.domain.etl.services.enricher_service:✅ BreakoutAnalyzer completado
INFO:src.domain.etl.services.enricher_service:✅ Stage 2 completado: 3 analyzers aplicados
INFO:src.domain.etl.services.enricher_service:🧹 Stage 3: Validación y limpieza
INFO:src.domain.etl.services.enricher_service:✅ Stage 3 completado
INFO:src.domain.etl.services.enricher_service:✅ TRANSFORM completado: 6→65 columnas
INFO:src.domain.etl.services.enricher_service:🔧 TRANSFORM: Starting transformation pipeline, for the next indicators: 
INFO:src.domain.etl.services.enricher_service:📊 Stage 1: Aplicando indicadores matemáticos (pandas-ta)
INFO:src.domain.etl.calculators.basic_indicators_calculator:🚀 IndicatorAnalyzer iniciando con 1450 registros
INFO:src.domain.etl.calculators.basic_indicators_calculator:🔍 No se especificaron indicadores, usando todos
INFO:src.domain.etl.calculators.basic_indicators_calculator:📈 Calculando indicadores de momentum...
INFO:src.domain.etl.calculators.basic_indicators_calculator:📊 Cal

Backtest.run:   0%|          | 0/1442 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1430 [00:00<?, ?bar/s]

...

Backtest.run:   0%|          | 0/1416 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1407 [00:00<?, ?bar/s]

Backtest.run:   0%|          | 0/1449 [00:00<?, ?bar/s]

INFO:src.infrastructure.database.repositories.symbol_strategy_repository:💾 Guardada mejor estrategia para AAPL.BA/1h: MACDStrategy (Sharpe=1.38, Ret=68.33%, DD=-20.60%)
INFO:src.infrastructure.database.repositories.symbol_strategy_repository:💾 Guardada mejor estrategia para TSLA.BA/1h: SMACrossoverWithRSIAndAvgVolStrategy (Sharpe=1.09, Ret=22.58%, DD=-10.57%)
INFO:src.infrastructure.database.repositories.symbol_strategy_repository:💾 Guardada mejor estrategia para MSFT.BA/1h: SMACrossoverStrategy (Sharpe=1.06, Ret=30.73%, DD=-20.26%)
INFO:src.infrastructure.database.repositories.symbol_strategy_repository:💾 Guardada mejor estrategia para NVDA.BA/1h: MACDStrategy (Sharpe=1.49, Ret=115.49%, DD=-13.30%)
INFO:src.infrastructure.database.repositories.symbol_strategy_repository:💾 Guardada mejor estrategia para BABA.BA/1h: SMACrossoverStrategy (Sharpe=1.18, Ret=88.55%, DD=-18.14%)
INFO:src.infrastructure.database.repositories.symbol_strategy_repository:💾 Guardada mejor estrategia para MELI.BA/


=== AAPL.BA ===
Backtest Report for MACDStrategy on AAPL.BA:
  Total Return: 68.33%
  Sharpe Ratio: 1.38
  Max Drawdown: -20.60%
  Win Rate: 52.17%
  Total Trades: 46
None

=== TSLA.BA ===
Backtest Report for SMACrossoverWithRSIAndAvgVolStrategy on TSLA.BA:
  Total Return: 22.58%
  Sharpe Ratio: 1.09
  Max Drawdown: -10.57%
  Win Rate: 38.10%
  Total Trades: 21
None

=== MSFT.BA ===
Backtest Report for SMACrossoverStrategy on MSFT.BA:
  Total Return: 30.73%
  Sharpe Ratio: 1.06
  Max Drawdown: -20.26%
  Win Rate: 48.81%
  Total Trades: 84
None

=== NVDA.BA ===
Backtest Report for MACDStrategy on NVDA.BA:
  Total Return: 115.49%
  Sharpe Ratio: 1.49
  Max Drawdown: -13.30%
  Win Rate: 48.94%
  Total Trades: 47
None

=== BABA.BA ===
Backtest Report for SMACrossoverStrategy on BABA.BA:
  Total Return: 88.55%
  Sharpe Ratio: 1.18
  Max Drawdown: -18.14%
  Win Rate: 44.57%
  Total Trades: 92
None

=== MELI.BA ===
Backtest Report for RSIStrategy on MELI.BA:
  Total Return: 51.91%
  Sharpe R